## Imports

In [23]:
import os
import re
import json
import pickle
import numpy as np
import pandas as pd
import torch
import faiss
from abc import ABC, abstractmethod
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore import InMemoryDocstore
from langchain.llms import Ollama
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate


## Abstract classes

### Preprocessing class

In [2]:
from abc import ABC, abstractmethod

class BasePreprocessor(ABC):
    def __init__(self):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=200,
            chunk_overlap=50, 
            length_function=lambda x: len(x.split()),
            separators=["\n\n\n", "\n\n", "\n", ". ", "! ", "? ", "; ", ", ", " ", ""],
            keep_separator=False,
            add_start_index=True,
            strip_whitespace=True
        )

    @abstractmethod
    def load_and_preprocess_data(self, file_path):
        pass

    @abstractmethod
    def process_documents_from_files(self, file_paths):
        pass


    def clean_text(self, text):
        return re.sub(r'\s+', ' ', re.sub(r'\n{3,}', '\n\n', str(text))).strip()



    def chunk_documents(self, individual_documents):
        chunked_docs = []
        for doc in individual_documents:
            chunks = self.text_splitter.split_text(doc.page_content)
            for i, chunk in enumerate(chunks):
                chunked_docs.append(
                    Document(
                        page_content=chunk,
                        metadata={
                            "pdf_id": doc.metadata["pdf_id"],
                            "chunk_id": i
                        }
                    )
                )
        print(f"✅ Total Chunks: {len(chunked_docs)}")
        return chunked_docs


In [3]:
class JSONPreprocessor(BasePreprocessor):
    def load_and_preprocess_data(self, file_path):
        with open(file_path, 'r') as f:
            raw_data = json.load(f)
        clean_texts = [self.clean_text(entry) for entry in raw_data if isinstance(entry, str)]
        return "\n".join(clean_texts)
    def process_documents_from_files(self, file_paths):
        documents = []

        for i, file_path in enumerate(file_paths):
            text = self.load_and_preprocess_data(file_path).strip()
            documents.append(
                Document(page_content=text, metadata={"pdf_id": i})
            )

        return documents


### Embeddings Abstract class

In [4]:
class Embedder(ABC): 
    def __init__(self, model_name, batch_size):
        self.model_name = model_name
        self.batch_size = batch_size
        
        self.device = (
            'cuda' if torch.cuda.is_available()
            else 'mps' if torch.backends.mps.is_available()
            else 'cpu'
        )
        self.embedding_model = HuggingFaceEmbeddings(model_name=model_name,model_kwargs={'device': self.device},encode_kwargs={'normalize_embeddings': True},multi_process=True,
                                                     show_progress=True,cache_folder='./embedder_model_cache')

    @abstractmethod
    def embed_documents(self, documents):
        pass

    @abstractmethod
    def batch_embed(self, texts, batch_size=None): 
        pass

class MultilingualEmbedder(Embedder): 
    def __init__(self, model_name, batch_size):
        super().__init__(model_name, batch_size)

    def embed_documents(self, documents):
        return self.batch_embed(documents, batch_size=self.batch_size)

    def batch_embed(self, texts, batch_size=None):
        if batch_size is None:
            batch_size = self.batch_size
        
        embeddings = []
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i + batch_size]
            batch_embeddings = self.embedding_model.embed_documents(batch)
            embeddings.extend(batch_embeddings)
        
        return np.array(embeddings, dtype=np.float32)


### Faiss Abstract class

In [ ]:
class VectorStoreBase(ABC):
    @abstractmethod
    def create_vector_store(self, documents, embedder_model):
        pass
    @abstractmethod
    def create_faiss_index(self, chunks_embed):
        pass
    @abstractmethod
    def search_faiss(self, query_embedding, top_k=5):
        pass
    @abstractmethod
    def search_chunks(self, query_embedding, top_k=5):
        pass
    @abstractmethod
    def save_faiss_index(self, file_index_name):
        pass
    @abstractmethod
    def load_faiss_index(self, file_index_name):
        pass




In [ ]:
class VectorStoreBase(ABC):
    @abstractmethod
    def create_vector_store(self, documents, embedder_model):
        pass
    
    @abstractmethod
    def get_relevant_documents(self, query, top_k=5):
        pass
    
    @abstractmethod
    def save_index(self, file_path):
        pass
    
    @abstractmethod
    def load_index(self, file_path):
        pass

class FAISS(VectorStoreBase):
    def __init__(self, embedder_model=None):
        self.index = None
        self.chunks_dict = None
        self.dimension = None
        self.total_vectors = 0
        self.index_type = "IndexFlatIP"
        self.embedder_model = embedder_model
    
    def create_vector_store(self, documents, embedder_model=None):
        """Create vector store from documents"""
        if embedder_model:
            self.embedder_model = embedder_model
        
        if not self.embedder_model:
            raise ValueError("Embedder model is required")
        
        texts = [doc.page_content for doc in documents]
        embeddings = self.embedder_model.batch_embed(texts)
        embeddings = np.array(embeddings).astype("float32")
        
        # Ensure embeddings are 2D
        if embeddings.ndim == 1:
            embeddings = embeddings.reshape(1, -1)
        
        self.dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatIP(self.dimension)
        self.index.add(embeddings)
        
        # Store text chunks with their indices
        self.chunks_dict = {i: text for i, text in enumerate(texts)}
        self.total_vectors = self.index.ntotal
        
        print(f"[FAISS] Created index with {self.total_vectors} vectors of dim {self.dimension}")
        return self
    
    def get_relevant_documents(self, query, top_k=5):
        """Main retriever function - returns LangChain Document objects"""
        if self.index is None:
            raise ValueError("Index not created. Call create_vector_store() first.")
        
        if not self.embedder_model:
            raise ValueError("Embedder model not set")
        
        # Get query embedding
        if isinstance(query, str):
            query_embedding = self.embedder_model.batch_embed([query])
            if isinstance(query_embedding, list) and len(query_embedding) > 0:
                query_embedding = query_embedding[0]
            elif isinstance(query_embedding, np.ndarray) and query_embedding.ndim > 1:
                query_embedding = query_embedding[0]
        else:
            query_embedding = self.embedder_model.batch_embed(query)
        
        # Search and format results
        results = self._search_chunks(query_embedding, top_k)
        
        return [
            Document(page_content=res['text'], metadata={"similarity": res['similarity']})
            for res in results
        ]
    
    def _search_chunks(self, query_embedding, top_k=5):
        """Internal search function - returns raw results"""
        # Ensure query_embedding is properly shaped
        query_embedding = np.array(query_embedding).astype("float32")
        
        # Handle different input shapes
        if query_embedding.ndim == 1:
            query_embedding = query_embedding.reshape(1, -1)
        elif query_embedding.ndim > 2:
            query_embedding = query_embedding.reshape(1, -1)
        
        print(f"[DEBUG] Query embedding final shape: {query_embedding.shape}")
        print(f"[DEBUG] Index dimension: {self.dimension}")
        
        # Verify dimensions match
        if query_embedding.shape[1] != self.dimension:
            raise ValueError(f"Query embedding dimension {query_embedding.shape[1]} doesn't match index dimension {self.dimension}")
        
        # Search FAISS index
        distances, indices = self.index.search(query_embedding, top_k)
        
        # Format results
        formatted = []
        for i in range(top_k):
            faiss_idx = indices[0][i]
            if faiss_idx != -1 and faiss_idx < len(self.chunks_dict):
                distance = distances[0][i]
                formatted.append({
                    'chunk_id': faiss_idx,
                    'text': self.chunks_dict[faiss_idx],
                    'distance': distance,
                    'similarity': float(distance)  # For cosine similarity, higher is better
                })
        
        return formatted
    
    def search_raw(self, query_embedding, top_k=5):
        """Search with raw embedding input - useful for advanced use cases"""
        return self._search_chunks(query_embedding, top_k)
    
    def save_index(self, file_path):
        """Save both FAISS index and metadata"""
        if self.index is None:
            raise ValueError("No index to save")
        
        # Save FAISS index
        faiss.write_index(self.index, f"{file_path}.faiss")
        
        # Save metadata
        metadata = {
            'chunks_dict': self.chunks_dict,
            'dimension': self.dimension,
            'total_vectors': self.total_vectors,
            'index_type': self.index_type
        }
        
        with open(f"{file_path}_metadata.pkl", 'wb') as f:
            pickle.dump(metadata, f)
        
        print(f"[FAISS] Index and metadata saved to {file_path}")
    
    def load_index(self, file_path, embedder_model=None):
        """Load both FAISS index and metadata"""
        if not os.path.exists(f"{file_path}.faiss"):
            raise FileNotFoundError(f"Index file {file_path}.faiss not found")
        
        if not os.path.exists(f"{file_path}_metadata.pkl"):
            raise FileNotFoundError(f"Metadata file {file_path}_metadata.pkl not found")
        
        # Load FAISS index
        self.index = faiss.read_index(f"{file_path}.faiss")
        
        # Load metadata
        with open(f"{file_path}_metadata.pkl", 'rb') as f:
            metadata = pickle.load(f)
        
        self.chunks_dict = metadata['chunks_dict']
        self.dimension = metadata['dimension']
        self.total_vectors = metadata['total_vectors']
        self.index_type = metadata['index_type']
        
        # Set embedder model if provided
        if embedder_model:
            self.embedder_model = embedder_model
        
        print(f"[FAISS] Index loaded: {self.total_vectors} vectors, dim {self.dimension}")
        return self
    
    def set_embedder_model(self, embedder_model):
        """Set or update the embedder model"""
        self.embedder_model = embedder_model
        return self
    
    def get_stats(self):
        """Get index statistics"""
        return {
            'total_vectors': self.total_vectors,
            'dimension': self.dimension,
            'index_type': self.index_type,
            'has_embedder': self.embedder_model is not None
        }



### LLM Abstract Class

In [7]:
class BaseLLM(ABC):
    def __init__(self, model_name, cache_folder):
        self.model_name = model_name
        self.cache_folder = cache_folder
        self.device = (
            'cuda' if torch.cuda.is_available()
            else 'mps' if torch.backends.mps.is_available()
            else 'cpu'
        )

    @abstractmethod
    def load_model(self):
        pass


class OLLAMA_LLM(BaseLLM):
    def __init__(self, model_name, cache_folder):
        super().__init__(model_name, cache_folder)

    def load_model(self):
        model = Ollama(model=self.model_name, temperature=0.3, num_ctx=4096)
        return model


class Hugging_Face_LLM(BaseLLM):
    def __init__(self, model_name, cache_folder):
        super().__init__(model_name, cache_folder)

    def load_model(self):
        tokenizer = AutoTokenizer.from_pretrained(
            self.model_name,
            cache_dir=self.cache_folder
        )
        model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            cache_dir=self.cache_folder,
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
            device_map="auto"  
        )
        return model, tokenizer

## Strategy Pattern Design

In [8]:
class TaskStrategy(ABC):
    @abstractmethod
    def run(self, input_text) :
        pass


In [ ]:
class ChattingStrategy(TaskStrategy):
    def __init__(self, llm, vector_store, embedder, top_k=5, return_sources=True):
        # Get actual LLM instance
        self.llm = llm
        self.vector_store = vector_store
        self.vector_store.set_embedder_model(embedder)
        self.top_k = top_k
        self.return_sources = return_sources
        self._build_chain()


    def format_docs(self, docs):
        return "\n\n".join(
            f"[Source {i} | PDF {doc.metadata.get('pdf_id', '?')}]: {doc.page_content}"
            for i, doc in enumerate(docs, 1)
        )

    def _build_chain(self):
        prompt_template = """You are a helpful assistant. Use the following context to answer the question.

            Context:
            {context}

            Question: {question}

            Please provide a comprehensive answer based on the context above. You MUST follow this exact format:

            RESPONSE:
            [Your main answer here]

            REASONING:
            [Explain your reasoning and how you used the context]

            SOURCES:
            [List the source numbers you referenced, for example: 1, 3, 5]
            """
        
        prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

        def retrieve_context(inputs):
            # Use the vector store directly 
            docs = self.vector_store.get_relevant_documents(inputs["question"], top_k=self.top_k)
            return self.format_docs(docs)

        self.chain = ({
                "context": RunnableLambda(retrieve_context), 
                "question": RunnablePassthrough()
            }
            | prompt
            | self.llm
            | StrOutputParser()
        )

    def parse_structured_response(self, response_text):
        cleaned_response = re.sub(r'<think>.*?</think>', '', response_text, flags=re.DOTALL)
        cleaned_response = re.sub(r'<[^>]+>', '', cleaned_response)
        cleaned_response = re.sub(r'\n\s*\n', '\n\n', cleaned_response.strip())

        sections = {'response': '', 'reasoning': '', 'sources': ''}
        current_section = None
        current_content = []

        lines = cleaned_response.split('\n')
        for line in lines:
            line = line.strip()
            if line.upper().startswith('RESPONSE:'):
                if current_section:
                    sections[current_section] = '\n'.join(current_content).strip()
                current_section = 'response'
                current_content = [line[9:].strip()]
            elif line.upper().startswith('REASONING:'):
                if current_section:
                    sections[current_section] = '\n'.join(current_content).strip()
                current_section = 'reasoning'
                current_content = [line[10:].strip()]
            elif line.upper().startswith('SOURCES:'):
                if current_section:
                    sections[current_section] = '\n'.join(current_content).strip()
                current_section = 'sources'
                current_content = [line[8:].strip()]
            elif current_section and line:
                current_content.append(line)

        if current_section:
            sections[current_section] = '\n'.join(current_content).strip()

        source_ids = [int(x) for x in re.findall(r'\d+', sections['sources'])] if sections['sources'] else []

        return {
            'answer': sections['response'],
            'reasoning': sections['reasoning'],
            'sources': source_ids,
            'raw_response': cleaned_response
        }

    def run(self, question):
        """Main method to run the chain and parse result."""
        
        response = self.chain.invoke({"question": question})
        print("Past chain call")
        print(f"Raw LLM response: {response}")  
        
        parsed = self.parse_structured_response(response)
        print("Past parser call")
        print(f"Parsed response: {parsed}")  
        
        if not self.return_sources:
            return parsed

    
        source_docs = self.vector_store.get_relevant_documents(question, top_k=self.top_k)
        parsed['source_documents'] = source_docs
        parsed['source_texts'] = [doc.page_content for doc in source_docs]
        return parsed



In [ ]:

class SummerizationStrategy(TaskStrategy):
    def __init__(self, llm, vector_store, embedder, top_k=5, return_sources=True):
        self.llm = llm
        self.vector_store = vector_store
        self.vector_store.set_embedder_model(embedder)
        self.top_k = top_k
        self.return_sources = return_sources
        self.prompt=rompt = ChatPromptTemplate.from_messages( [("system", "Write a concise summary of the following:\\n\\n{context}")])
    
    def run(self,document):
        chain = create_stuff_documents_chain(self.llm, self.prompt)
        result = chain.invoke({"context": [document]})
        print(result)








In [30]:

class QuestionStrategy(TaskStrategy):
    def __init__(self, llm, vector_store, embedder, top_k=5, return_sources=True):
        self.llm = llm
        self.vector_store = vector_store
        self.vector_store.set_embedder_model(embedder)
        self.top_k = top_k
        self.return_sources = return_sources
        self.prompt= ChatPromptTemplate.from_template(
            """You are a helpful assistant tasked with generating question-answer pairs for study purposes.

                Text:
                {context}

                Generate 2-3 meaningful questions and their answers based only on the above text. 

                IMPORTANT: Format your output exactly as shown below with no additional text, explanations, or formatting:

                Q1: [question text]
                A1: [answer text]

                Q2: [question text]
                A2: [answer text]

                Q3: [question text]
                A3: [answer text]

                Rules:
                - Use only information from the provided text
                - Do not invent facts or add external knowledge
                - Each question should be on a single line
                - Each answer should be on a single line
                - Do not include any text before the first Q1 or after the last answer
                - Do not use bullet points, numbering, or other formatting
                - Always """)

    def parse_qa_pairs(self,qa_output):
        """Parse the QA output into structured format"""
        qa_pairs = []
        
        # Split by lines and process
        lines = qa_output.strip().split('\n')
        
        i = 0
        while i < len(lines):
            line = lines[i].strip()
            
            # Look for question pattern
            q_match = re.match(r'Q(\d+):\s*(.+)', line)
            if q_match and i + 1 < len(lines):
                q_num = q_match.group(1)
                question = q_match.group(2).strip()
                
                # Look for corresponding answer
                next_line = lines[i + 1].strip()
                a_match = re.match(f'A{q_num}:\\s*(.+)', next_line)
                
                if a_match:
                    answer = a_match.group(1).strip()
                    qa_pairs.append({
                        'question': question,
                        'answer': answer
                    })
                    i += 2  # Skip both Q and A lines
                else:
                    i += 1
            else:
                i += 1
        
        return qa_pairs
    def run(self,docs, batch_size=2):
        """Generate QA pairs from a list of Document chunks"""
        qa_pairs = []
        qa_chain = self.prompt | self.llm | StrOutputParser()
        for doc in docs:
            try:
                # Generate QA output
                qa_output = qa_chain.invoke({"context": doc.page_content})
                
                # Parse the output into structured format
                parsed_qa = self.parse_qa_pairs(qa_output)
                
                qa_pairs.append({
                    "pdf_id": doc.metadata.get("pdf_id"),
                    "chunk_id": doc.metadata.get("chunk_id"),
                    "text": doc.page_content,
                    "qa_output": qa_output,  # Keep raw output for debugging
                    "parsed_qa": parsed_qa   # Add parsed structured data
                })
                
                print(f"✅ Generated {len(parsed_qa)} QA pairs for chunk {doc.metadata.get('chunk_id', 'unknown')}")
            
            except Exception as e:
                print(f"❌ Failed to generate QA for chunk {doc.metadata}: {e}")

        return qa_pairs





## Classes Testing

In [10]:
# Testing cell
paths=["Market Research Report_extracted_text.json", 'PMS Market Research_extracted_text.json']
docs=JSONPreprocessor()
data=docs.process_documents_from_files(paths)
individual_documents = [ Document(page_content=pdf.page_content, metadata={"pdf_id": i})
    for i, pdf in enumerate(data) if pdf.page_content
]
chunked_docs=docs.chunk_documents(individual_documents)

✅ Total Chunks: 71


In [11]:
multilingual_embedder=MultilingualEmbedder(model_name="sentence-transformers/all-MiniLM-L6-v2", batch_size=32)

In [12]:
multilingual_embedder.batch_size

32

In [13]:
llm=OLLAMA_LLM('qwen3:8b','llm_cache').load_model()

/var/folders/c2/f9lh6rmd4q1648_pfl1636zw0000gn/T/ipykernel_11398/3805475503.py:21: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model=self.model_name, temperature=0.3, num_ctx=4096)


In [14]:
faiss_store = FAISS()
faiss_store.set_embedder_model(multilingual_embedder)
faiss_store.create_vector_store(chunked_docs)

[FAISS] Created index with 71 vectors of dim 384


In [19]:
strategy = ChattingStrategy(llm, faiss_store, multilingual_embedder,top_k=5)


In [20]:
response = strategy.run("What is the carbon footprint of solar panels?")

[DEBUG] Query embedding final shape: (1, 384)
[DEBUG] Index dimension: 384
Past chain call
Raw LLM response: <think>
Okay, the user is asking about the carbon footprint of solar panels. Let me check the provided context to see if there's any information related to that.

Looking through the sources, Source 1 talks about renewable energy platforms and solar-focused systems but doesn't mention carbon footprints. Source 2 is about project management systems for PV projects, so no relevant info there. Source 3 seems to be about pricing for a service, not related. Source 4 discusses project management features for solar projects, like document control and collaboration, but again, no carbon footprint data. Source 5 mentions Oracle Aconex pricing and features, which also isn't related.

None of the sources provide data on the carbon footprint of solar panels. The context is focused on project management tools and systems for solar projects, not environmental impact metrics. Therefore, I can'

In [21]:
response

{'answer': 'The provided context does not contain any information about the carbon footprint of solar panels. The sources focus on project management systems, monitoring tools, and pricing details for renewable energy and solar PV projects, but none address environmental impact metrics such as carbon footprint.',
 'reasoning': 'The question about the carbon footprint of solar panels is unrelated to the context provided. The sources discuss technical tools for project management, document control, and system pricing, but none mention environmental metrics, lifecycle analysis, or carbon emissions associated with solar panels.',
 'sources': [1, 2, 3, 4, 5],
 'raw_response': 'RESPONSE:\nThe provided context does not contain any information about the carbon footprint of solar panels. The sources focus on project management systems, monitoring tools, and pricing details for renewable energy and solar PV projects, but none address environmental impact metrics such as carbon footprint.\n\nREAS

In [22]:
print(response['answer'])

The provided context does not contain any information about the carbon footprint of solar panels. The sources focus on project management systems, monitoring tools, and pricing details for renewable energy and solar PV projects, but none address environmental impact metrics such as carbon footprint.


In [25]:
summerization=SummerizationStrategy(llm, faiss_store, multilingual_embedder,top_k=5)


In [29]:
for doc in individual_documents:
    summerization.run(doc)

<think>
Okay, I need to create a concise summary of this market research report on document translation tools. Let me start by reading through the entire text again to make sure I understand the key points.

The report evaluates several tools: Doctranslator, Doctranslate.io, TranslaDocs, SmallPDF, Doclingo, and DeepL. They tested these tools for handling Arabic, French, and English, focusing on layout preservation, OCR support, translation accuracy, and pricing. The test cases included text-based documents, scanned documents, tables (text and image), and documents with stamps/signatures.

The key findings mention that no single tool meets all requirements. Doctranslator is good for English to Arabic but lacks OCR. Doctranslate.io has OCR but poor layout and handles Arabic numerals poorly. TranslaDocs doesn't support OCR or Arabic. SmallPDF has limited translation. Doclingo does OCR well for Arabic to English but struggles with image tables. DeepL is great for English OCR but no Arabic 

In [31]:
Question=QuestionStrategy(llm, faiss_store, multilingual_embedder,top_k=5)

In [33]:
Question.run(individual_documents)

✅ Generated 3 QA pairs for chunk unknown
✅ Generated 3 QA pairs for chunk unknown


[{'pdf_id': 0,
  'chunk_id': None,
  'text': "MARKET RESEARCH REPORT: ANALYSIS OF DOCUMENT TRANSLATION TOOLS Evaluating Leading Solutions for Multilingual Document Translation Mah inour Mohammad\nIntroduction This market research report analyzes competitors offering document translation tools that support PDF, Word, Excel, and scanned images while preserving layout and formatting. The focus is on tools that handle Arabic, French, and English languages, catering to both B2B and B2C markets. The key features evaluated include layout preservation, Arabic support and quality, translation accuracy and speed, pricing model, and Optical Character Recognition (OCR) support. To assess these tools, a series of test cases were conducted for each language, including: 1. Text -based documents: Evaluating basic translation accuracy, layout preservation, handling of number lists, bullet points, and right -to-left (RTL) and left -to-right (LTR) conversions. 2. Scanned documents: Testing OCR performanc